<a href="https://colab.research.google.com/github/deniskapel/GenerativeChitChat/blob/master/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
mkdir data
mkdir pretrained
wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1uSgX8EaXtSR1yZgs-pJGYiZXi7tPEBrE' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1uSgX8EaXtSR1yZgs-pJGYiZXi7tPEBrE" -O data/qa_data.jsonl.zip && rm -rf /tmp/cookies.txt
unzip data/qa_data.jsonl.zip -d data

In [2]:
%%bash
wget "https://raw.githubusercontent.com/deniskapel/GenerativeChitChat/master/requirements.txt"
pip install -r requirements.txt

  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
  Attempting uninstall: seaborn
    Found existing installation: seaborn 0.11.2
    Uninstalling seaborn-0.11.2:
      Successfully uninstalled seaborn-0.11.2
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.1
    Uninstalling scikit-learn-1.0.1:
      Successfully uninstalled scikit-learn-1.0.1
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


--2021-12-25 15:26:07--  https://raw.githubusercontent.com/deniskapel/GenerativeChitChat/master/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123 [text/plain]
Saving to: ‘requirements.txt’

     0K                                                       100% 2.71M=0s

2021-12-25 15:26:07 (2.71 MB/s) - ‘requirements.txt’ saved [123/123]

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.9.1 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.9.1 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but

In [3]:
import codecs
import json
import random
import math
import numpy as np
import pandas as pd

import random
import json
import torch
from torch.utils.data import Dataset, DataLoader, Sampler
from torch.nn.utils.rnn import pad_sequence
from torch import nn
import torch.nn.functional as F

from tqdm.notebook import tqdm

from matplotlib import pyplot as plt

import youtokentome as yttm

In [4]:
assert torch.cuda.is_available(), 'no gpu available'
device = torch.device('cuda')
device

device(type='cuda')

In [5]:
%%bash
wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1QALzCQV3awJ84_LAVoo1FTdZ65VHX8Mc' -O pretrained/my_pretrained_bpe_lm.model

--2021-12-25 15:27:55--  https://docs.google.com/uc?export=download&id=1QALzCQV3awJ84_LAVoo1FTdZ65VHX8Mc
Resolving docs.google.com (docs.google.com)... 74.125.142.139, 74.125.142.101, 74.125.142.100, ...
Connecting to docs.google.com (docs.google.com)|74.125.142.139|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0k-9g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/se104gbi4rimgfg9kgrtijfdvbc3asea/1640446050000/17057554186329209310/*/1QALzCQV3awJ84_LAVoo1FTdZ65VHX8Mc?e=download [following]
--2021-12-25 15:27:56--  https://doc-0k-9g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/se104gbi4rimgfg9kgrtijfdvbc3asea/1640446050000/17057554186329209310/*/1QALzCQV3awJ84_LAVoo1FTdZ65VHX8Mc?e=download
Resolving doc-0k-9g-docs.googleusercontent.com (doc-0k-9g-docs.googleusercontent.com)... 74.125.195.132, 2607:f8b0:400e:c09::84
Connecting to doc-0k-9g-docs.googleusercontent.com (doc-0k-9g-d

In [6]:
tokenizer = yttm.BPE(model="pretrained/my_pretrained_bpe_lm.model")

In [7]:
batch_size = 64
pad_index = 0
eos_index = 3
vocab_size=30_000

## Data

In [8]:
!sed 5q data/qa_data.jsonl

{"question": "долго ли идут деньги с яндексденег на карту visa?", "category": "Бизнес, Финансы", "responses": ["нет. прорыв 35 ;)"]}
{"question": "можно ли зарегистрировать авто в другом регионе", "category": "Авто, Мото", "responses": ["можно на родственника из того региона.. .  а потом ездить по доверке"]}
{"question": "что делать если у меня очень тонкие ногти а хочется их отрастить?", "category": "Красота и Здоровье", "responses": ["витамины и умная эмаль (каждый день)", "ванночки с морской солью. с вечера мажь ногти сверху йодом. не бойся, до утра все впитается.", "умная эмаль, витамины, йод, и поменьше крась лаком ", "лаки фирмы trind производство usa + кальций"]}
{"question": "в чем отличие медитации от йоги?", "category": "Спорт", "responses": ["букв в йоге меньше", "в медитации ты просто сидишь и мммммычишь. а в йоге всяко разные упражнения вытворяешь", "в медитации вроде просто тупо сидишь и успокаеваешься, а в йоге еще и ноги за уши закидывать надо"]}
{"question": "когда нач

In [9]:
with codecs.open("data/qa_data.jsonl", encoding='utf-8-sig') as reader:
    lines = reader.read().split("\n")
    lines = list(map(json.loads, filter(None, lines)))

data = []
for line in tqdm(lines):
    for response in line['responses']:
        data.append(
            {'question': line['question'].lower(),
             'category': line['category'],
             'response': response.lower()})

del lines
df = pd.json_normalize(data)
del data

  0%|          | 0/2808811 [00:00<?, ?it/s]

In [10]:
df.head()

,question,category,response
0,долго ли идут деньги с яндексденег на карту visa?,"Бизнес, Финансы",нет. прорыв 35 ;)
1,можно ли зарегистрировать авто в другом регионе,"Авто, Мото",можно на родственника из того региона.. . а п...
2,что делать если у меня очень тонкие ногти а хо...,Красота и Здоровье,витамины и умная эмаль (каждый день)
3,что делать если у меня очень тонкие ногти а хо...,Красота и Здоровье,ванночки с морской солью. с вечера мажь ногти ...
4,что делать если у меня очень тонкие ногти а хо...,Красота и Здоровье,"умная эмаль, витамины, йод, и поменьше крась л..."


In [11]:
df_mini = df.sample(frac=0.01, random_state=42)
train_df = df_mini.sample(frac=0.8,random_state=42)
val_df = df_mini.drop(train_df.index)
test_df = val_df.sample(frac=0.5,random_state=42)
val_df = val_df.drop(test_df.index)

In [12]:
val_df.category.value_counts()

Знакомства, Любовь, Отношения    1037
Философия, Непознанное            865
Семья, Дом, Дети                  760
Искусство и Культура              630
Красота и Здоровье                630
Авто, Мото                        573
Спорт                             521
Животные, Растения                499
Досуг, Развлечения                428
Еда, Кулинария                    382
Стиль, Мода, Звезды               367
Бизнес, Финансы                   302
Путешествия, Туризм               268
Работа, Карьера                   258
Образование                       247
Name: category, dtype: int64

### Datasets

In [13]:
class QAData(torch.utils.data.Dataset):
    
    def __init__(self, data: pd.DataFrame, tokenizer,
                 pad_index=0, eos_index=3, response_len=32):
        
        self.x = data.question.tolist()
        self.y = data.response.tolist()
        self.tokenizer = tokenizer
        # to use with beam search later
        self.categories = data.category.tolist()
        self.response_maxlen = response_len
        self.pad_index = pad_index
        self.eos_index = eos_index
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        """ x is a question and y is an asnwer """
        x = self.tokenizer.encode(self.x[index], bos=True, eos=True)
        y = self.tokenizer.encode(self.y[index], bos=True, eos=True)

        # different shape of samples is handled by batch_sampler and encoder/decoder
        x = torch.Tensor(x).type(torch.int)
        y = torch.Tensor(y).type(torch.int)
        
        return x, y 

    def collate_batch(self, batch):
        """
        add padding to dynamically match the longest sample in a batch
        """
        batch_x, batch_y = self.__group_samples(batch)

        batch_x = pad_sequence(batch_x, padding_value=self.pad_index, batch_first=True)
        batch_y = pad_sequence(batch_y, padding_value=self.pad_index, batch_first=True)

        return batch_x, batch_y

    def __group_samples(self, batch: list) -> list:
        """ 
        input: [(sample_1_x, sample_1_y), (sample_2_x, sample_2_y)]
        output: [[samle_1_x, sample_2_x], [sample_1_y, sample_2_y]]
        """
        batch_x = []
        batch_y = []
        for sample in batch:
            batch_x.append(sample[0])
            batch_y.append(sample[1])

        return batch_x, batch_y

In [14]:
train_dataset = QAData(
    data=train_df,
    tokenizer=tokenizer)

val_dataset = QAData(
    data=val_df,
    tokenizer=tokenizer)

test_dataset = QAData(
    data=test_df,
    tokenizer=tokenizer)

len(train_dataset), len(val_dataset), len(test_dataset)

(62137, 7767, 7767)

### DataLoader

In [15]:
class Sampler():
    def __init__(self, dataset, batch_size=64):
        self.dataset = dataset
        self.n_batches = len(dataset) // batch_size
        self.batch_size = batch_size

    def __iter__(self):
        indices = [(i, len(s[0])) for i, s in enumerate(self.dataset)]
        random.shuffle(indices)
        pooled_indices = []
        # create pool of indices with similar lengths 
        for i in range(0, len(indices), batch_size * 100):
            pooled_indices.extend(
                sorted(indices[i:i + batch_size * 100], key=lambda x: x[1])
                )
        pooled_indices = [x[0] for x in pooled_indices]
        
        # yield indices for current batch
        for i in range(0, len(pooled_indices), batch_size):
            yield pooled_indices[i:i + batch_size]

train_sampler = Sampler(train_dataset)
val_sampler = Sampler(val_dataset)
test_sampler = Sampler(test_dataset)

In [16]:
train_loader = DataLoader(
    train_dataset,
    collate_fn=train_dataset.collate_batch,
    batch_sampler=train_sampler)

val_loader = DataLoader(
    val_dataset,
    collate_fn=val_dataset.collate_batch,
    batch_sampler=val_sampler)

test_loader = DataLoader(
    test_dataset,
    collate_fn=val_dataset.collate_batch,
    batch_sampler=test_sampler)

In [33]:
for batch in train_loader:
    continue
    # break

batch[0].shape, batch[1].shape

(torch.Size([57, 41]), torch.Size([57, 32]))

In [18]:
train_sampler.n_batches,val_sampler.n_batches,test_sampler.n_batches

(970, 121, 121)

In [19]:
progress_bar = tqdm(total=len(train_loader.dataset), desc='Testing')

for x, y in train_loader:
    progress_bar.update(x.size(0))
    
progress_bar.close()

Testing:   0%|          | 0/62137 [00:00<?, ?it/s]

In [20]:
progress_bar = tqdm(total=len(val_loader.dataset), desc='Testing')

for x, y in val_loader:
    progress_bar.update(x.size(0))
    
progress_bar.close()

Testing:   0%|          | 0/7767 [00:00<?, ?it/s]

In [21]:
progress_bar = tqdm(total=len(test_loader.dataset), desc='Testing')

for x, y in test_loader:
    progress_bar.update(x.size(0))
    
progress_bar.close()

Testing:   0%|          | 0/7767 [00:00<?, ?it/s]

In [22]:
batch[0].shape, batch[1].shape

(torch.Size([64, 5]), torch.Size([64, 29]))

## Model

### Encoder/Decoder

In [62]:
class Encoder(nn.Module):
    def __init__(self, emb_dim, enc_hs, dec_hs, 
                 dropout=0, vocab_size=30_000, padding_idx=0):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=vocab_size, 
                                      embedding_dim=emb_dim,
                                      padding_idx=padding_idx)
        self.rnn = nn.GRU(emb_dim, enc_hs, bidirectional = True, batch_first=True)
        self.fc = nn.Linear(enc_hs * 2, dec_hs)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden=None):
        embedded = self.embedding(input)
        outputs, hidden = self.rnn(embedded, hidden)
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        return outputs, hidden

class Attention(nn.Module):
    def __init__(self, enc_hs, dec_hs):
        super().__init__()
        
        self.attn = nn.Linear((enc_hs * 2) + dec_hs, dec_hs)
        self.v = nn.Linear(dec_hs, 1, bias = False)
        
    def forward(self, encoder_outputs, hidden):
        batch_size = encoder_outputs.shape[0]
        src_len = encoder_outputs.shape[1]
        
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        attention = self.v(energy).squeeze(2)
        return F.softmax(attention, dim=1)


class Decoder(nn.Module):
    def __init__(self, emb_dim, enc_hs, dec_hs, 
                 attention, dropout=0, 
                 vocab_size=30_000, padding_idx=0):
        super().__init__()
        self.output_dim = vocab_size
        self.attention = attention
        self.embedding = nn.Embedding(num_embeddings=vocab_size, 
                                      embedding_dim=emb_dim,
                                      padding_idx=padding_idx)
        self.rnn = nn.GRU((enc_hs * 2) + emb_dim, dec_hs)
        self.fc_out = nn.Linear((enc_hs * 2) + dec_hs + emb_dim, vocab_size)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, encoder_outputs, hidden):
        input = input.unsqueeze(0)
        embedded = self.embedding(input)
        print(encoder_outputs.shape)
        a = self.attention(encoder_outputs, hidden)
        a = a.unsqueeze(1)

        weighted = torch.bmm(a, encoder_outputs)
        weighted = weighted.permute(1, 0, 2)
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        return prediction, hidden.squeeze(0)

In [63]:
encoder = Encoder(emb_dim=512, enc_hs = 128, dec_hs=128)
attn = Attention(enc_hs = 128, dec_hs=128)
decoder = Decoder(emb_dim=512, enc_hs = 128, dec_hs=128,
                  attention=Attention(enc_hs = 128, dec_hs=128))

In [64]:
output, hidden = encoder.forward(input=batch[0])
att_energy = attn.forward(output, hidden=hidden)
dec_output, dec_hidden = decoder.forward(batch[1][:,0], output, hidden)
dec_output, dec_hidden = decoder.forward(batch[1][:,1], output, hidden)

torch.Size([57, 41, 256])
torch.Size([57, 41, 256])


In [54]:
output.shape, hidden.shape, att_energy.shape, dec_output.shape, batch[1][:,1].shape

(torch.Size([57, 41, 256]),
 torch.Size([57, 128]),
 torch.Size([57, 41]),
 torch.Size([57, 30000]),
 torch.Size([57]))

In [75]:
class QAmodel(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, questions, responses, teacher_forcing_ratio = 0.5):
        
        batch_size = questions.shape[0]
        trg_len = responses.shape[1]
        trg_vocab_size = self.decoder.output_dim
        
        # create dummy tensor to store output sequence and put in to gpu
        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)
        
        encoder_outputs, hidden = self.encoder(questions)
                
        #first input to the decoder is the <sos> tokens
        input = responses[:,0]
        for t in range(1, trg_len):
            output, hidden = self.decoder(input, encoder_outputs, hidden)
            print(output.shape, outputs.shape)
            outputs[:,t,:] = output
            input = output.argmax(1) 

        return outputs

ENC_EMB_DIM = 128
DEC_EMB_DIM = 128
ENC_HID_DIM = 128
DEC_HID_DIM = 128

attention_layer = Attention(ENC_HID_DIM, DEC_HID_DIM)
encoder = Encoder(ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM)
decoder = Decoder(DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, attention_layer)

In [ ]:
model = QAmodel(encoder, decoder, 'cpu')

with torch.no_grad():
    pred = model.forward(batch[0], batch[1])

pred.shape

In [450]:
model = QAmodel(encoder, decoder, device)
model.to(device)

QAmodel(
  (encoder): Encoder(
    (embedding): Embedding(30000, 256, padding_idx=0)
    (rnn): GRU(256, 512, batch_first=True, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(30000, 256, padding_idx=0)
    (rnn): GRU(1280, 512)
    (fc_out): Linear(in_features=1792, out_features=30000, bias=True)
    (dropout): Dropout(p=0, inplace=False)
  )
)

In [77]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'Количество обучаемых параметров в сети: {count_parameters(model):,}')

Количество обучаемых параметров в сети: 23,547,824


## Train

In [ ]:
def train(model, loader, criterion, optimizer, clip=3., last_n_losses=500, verbose=True):

    losses = []
    progress_bar = tqdm(total=len(loader.dataset), disable=not verbose, desc='Train')
    model.train()
    for x, y in loader:
        
        x = x.to(device)
        y = y.to(device)
        
        

        loss = criterion(pred.view(-1, pred.size(-1)), y.view(-1))

        optimizer.zero_grad()
        loss.backward()
        # рассчитали градиенты и клипаем их
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()

        losses.append(loss.item())

        progress_bar.set_postfix(loss=np.mean(losses[-last_n_losses:]),
                                 perplexity=np.exp(np.mean(losses[-last_n_losses:])))

        progress_bar.update()

    progress_bar.close()
    
    return losses